# Tache 1 : Person TABLE

#### 1.1 Extraction des données  
Chargement des fichiers


*   ir_ben_r : Table contenant les informations des assurés.
*   er_prs_f : Table contenant les informations sur les prestations remboursées.
*   T_mcoaaE : Table contenant des informations sur les établissements de santé
*   ir_act_v et ir_spe_v : Tables contenant des informations sur les professionnels de santé.

In [54]:
import pandas as pd
import os

# Load the data files
ir_ben_r = pd.read_csv("./ir_ben_r (1).csv")
er_prs_f = pd.read_csv("./er_prs_f (1).csv")
T_mcoaae = pd.read_csv("./t_mcoaae (1).csv")
ir_act_v = pd.read_csv("./ir_act_v (1).csv")
ir_spe_v = pd.read_csv("./ir_spe_v (1).csv")

# Check if the DataFrames were loaded correctly
print(ir_ben_r.head())
print(er_prs_f.head())
print(T_mcoaae.head())
print(ir_act_v.head())
print(ir_spe_v.head())

                   NUM_ENQ  ben_sex_cod  ben_nai_ann  ben_nai_moi  \
0  DPXX:00000000000000001X            2         1963           12   
1    DPXX:000000000000002X            1         1971            2   
2    DPXX:000000000000003X            1         1962           12   
3    DPXX:000000000000004X            2         1959            3   
4    DPXX:000000000000005X            1         1998            4   

   ben_res_dpt  ben_res_reg  
0           75          114  
1           93          114  
2           93          114  
3           94          114  
4           93          114  
   id                         NUM_ENQ  prs_nat_ref exe_soi_dtd exe_soi_dtf  \
0   1  DPXX:000000000000000000000001X         1130  2013-03-04  2013-03-04   
1   2  DPXX:000000000000000000000001X         1331  2013-03-05  2013-03-05   
2   3  DPXX:000000000000000000000001X         3313  2013-03-05  2013-03-05   
3   4  DPXX:000000000000000000000001X         3125  2013-03-07  2013-03-07   
4   5  DPXX:000

In [26]:
import pandas as pd
import re

# 2. Ajout de la colonne Person Id

# Extraction de la partie numérique du NUM_ENQ
def extract_numeric_id(num_enq):
    # Trouver tous les chiffres après le préfixe
    numeric_part = re.search(r"(?<=:)\d+", num_enq).group()
    return int(numeric_part)  # Conversion en entier


Résultat :
Empty DataFrame
Columns: []
Index: []


In [42]:
import pandas as pd
import hashlib
from datetime import datetime

# Creation du code_commune
ir_ben_r['code_commune'] = (ir_ben_r['ben_res_dpt'].astype(str)[:2] +
                            ir_ben_r['ben_res_reg'].astype(str)).astype(int)

ir_ben_r['location_id'] = ir_ben_r['code_commune'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())

# 2. Création de la table PERSON OMOP
Person = pd.DataFrame({
    "person_id": ir_ben_r["NUM_ENQ"].apply(extract_numeric_id),
    "gender_concept_id": ir_ben_r["ben_sex_cod"].map({1: 8507, 2: 8532}),
    "year_of_birth": ir_ben_r["ben_nai_ann"],
    "month_of_birth": ir_ben_r["ben_nai_moi"],
    "person_source_value": ir_ben_r["NUM_ENQ"],
    "gender_source_value": ir_ben_r["ben_sex_cod"],
    "location_id": ir_ben_r["location_id"]
})


# 3. Création de la Table LOCATION
Location = pd.DataFrame({
    "location_source_value":ir_ben_r["code_commune"] ,
}).drop_duplicates()

# Create location_id as hash of location_source_value
Location['location_id'] = Location['location_source_value'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())


# Export
Person.to_csv("omop_person.csv", index=False)
Location.to_csv("omop_location.csv", index=False)

print(Person.head())
print(Location.head())

   person_id  gender_concept_id  year_of_birth  month_of_birth  \
0          1               8532           1963              12   
1          2               8507           1971               2   
2          3               8507           1962              12   
3          4               8532           1959               3   
4          5               8507           1998               4   

       person_source_value  gender_source_value  \
0  DPXX:00000000000000001X                    2   
1    DPXX:000000000000002X                    1   
2    DPXX:000000000000003X                    1   
3    DPXX:000000000000004X                    2   
4    DPXX:000000000000005X                    1   

                                         location_id  
0  3726ea216853610b03eb430297a2a7fb6a002d0c42ebe3...  
1  8cb624ac1c63fc8494076b622cdf3bb0278c6e1bd47f74...  
2  8cb624ac1c63fc8494076b622cdf3bb0278c6e1bd47f74...  
3  aba4b4460e4771b9a62c83ac2734787ee87ab095165720...  
4  8cb624ac1c63fc8494

# Tache 2 : CARE_SITE TABLE

In [ ]:
# 3. Création de la Table LOCATION
Location = pd.DataFrame({
    "location_source_value":ir_ben_r["code_commune"] ,
}).drop_duplicates()

# Create location_id as hash of location_source_value
Location['location_id'] = Location['location_source_value'].apply(lambda x: hashlib.sha256(str(x).encode()).hexdigest())




In [48]:
# prompt: # create sqlite database databse.sql
# and create a table ( CARE site de OMOP

import sqlite3
# Create a connection to the SQLite database
conn = sqlite3.connect('database.sql')
cursor = conn.cursor()

# Create the CARE_SITE table
cursor.execute('''
  CREATE TABLE IF NOT EXISTS CARE_SITE (
    care_site_id INTEGER PRIMARY KEY AUTOINCREMENT,
    care_site_name VARCHAR,
    location_id INTEGER,
    care_site_source_value VARCHAR
  )
''')

# Commit the changes and close the connection
conn.commit()



In [49]:

# Data from T_mcoaae
for index, row in T_mcoaae.iterrows():
  care_site_source_value = row['eta_num']
  care_site_name = row['soc_rai']
  location_id = str(row['eta_num'])[0:2]

  cursor.execute('''
    INSERT INTO CARE_SITE (care_site_name, care_site_source_value, location_id)
    VALUES (?, ?, ?)
  ''', (care_site_name, care_site_source_value, location_id))

# Commit the changes and close the connection
conn.commit()
conn.close()



# Tache 2 : CARE SITE TABLE

In [50]:
# prompt: create an empty apache spark table called provider de omop add define the colomns

!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# Initialize Spark session
spark = SparkSession.builder.appName("OMOPProvider").getOrCreate()

# Define the schema for the provider table
provider_schema = StructType([
    StructField("provider_id", IntegerType(), True),
    StructField("specialty_concept_id", IntegerType(), True),
    StructField("provider_source_value", StringType(), True),
    StructField("specialty_source_value", StringType(), True)
])

# Create an empty DataFrame with the defined schema
provider_df = spark.createDataFrame([], provider_schema)

# Create the table in Spark
provider_df.createOrReplaceTempView("provider")

#Verify
spark.sql("DESCRIBE provider").show()


+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|         provider_id|      int|   NULL|
|specialty_concept_id|      int|   NULL|
|provider_source_v...|   string|   NULL|
|specialty_source_...|   string|   NULL|
+--------------------+---------+-------+



In [52]:
# prompt: range for  IR_SPE_V read pfs_spe_cod and inset into spark.sql("""
# INSERT INTO provider  ( do not recreate anything do not import anything

# Assuming spark session is already initialized and the 'provider' table is created

# Iterate through each row of ir_spe_v
for index, row in ir_spe_v.iterrows():
    pfs_spe_cod = row['pfs_spe_cod']

    # Check if the code is within the desired range
    if 1 <= int(pfs_spe_cod) <= 999:

      # Define the SQL query to insert into the 'provider' table
      # Create a provider id to make it unique
      provider_id= int(str(row['pfs_spe_cod'])+str(index))

      # Using row to get the provider_source_value
      provider_source_value = row['spe_cod']
      specialty_source_value= row['pfs_spe_cod']

      insert_query = f"""
        INSERT INTO provider (provider_id, specialty_concept_id, provider_source_value, specialty_source_value)
        VALUES ({provider_id}, {pfs_spe_cod}, '{provider_source_value}', '{specialty_source_value}')
      """

      # Execute the SQL query
      spark.sql(insert_query)


KeyError: 'spe_cod'